In [1]:
:set -XOverloadedStrings

import Control.Monad (unless)
import Control.Monad.Trans.State.Strict as SS
import Data.Attoparsec.ByteString
import Data.Byteable
import qualified Data.Map.Strict        as M
import qualified Data.ByteString.Lazy   as L
import qualified Data.ByteString        as B
import qualified Codec.Compression.Zlib as Z
import Pipes
import Prelude hiding (take)
import qualified Pipes.ByteString as PB
import qualified Pipes.Prelude as PP
import qualified Pipes.Zlib as PZ
import qualified Pipes.Attoparsec as PA
import qualified System.IO as SI

import Duffer.Pack
import Duffer.Pack.Parser
import Duffer.Pack.Entries
import Duffer.Pack.Streaming

In [2]:
compressedContent = L.toStrict $ Z.compress "compressed content"
uncompressedContent = L.toStrict $ "uncompressed content"
mingled = B.concat [compressedContent, uncompressedContent, compressedContent]
mingledProducer = yield mingled
decompressed = PZ.decompress' PZ.defaultWindowBits mingledProducer
(Right (value, restProducer)) <- next decompressed
value
(Left (Left finalProducer)) <- next restProducer
(Right (final, _)) <- next finalProducer
moreCompressed = B.drop (B.length uncompressedContent) final
decompressed' = PZ.decompress' PZ.defaultWindowBits (yield moreCompressed)
(Right (value', restProducer')) <- next decompressed'
value'
:t restProducer'

"compressed content"

"compressed content"

restProducer' :: Producer ByteString IO (Either (Producer ByteString IO ()) ())

In [19]:
import qualified Prelude as P

content  <- B.readFile ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.pack"
((start, count), entries) <- runStateT parsePackfileStart (yield content)
result <- loopEntries entries start 832 M.empty
rest = snd result
:t rest
(header, r, decompressedP, level) <- evalStateT getNextEntry rest
level
:t decompressedP
:t next decompressedP
streamedMap = fst result
M.keys $ fst result
iBSMap <- indexedByteStringMap ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.idx"
iEMap  <- indexedEntryMap ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.idx"
iEMap M.! 2484
P.take 832 $ M.keys iBSMap
(streamedMap M.! 2285) == (iBSMap M.! 2285)

rest :: Producer ByteString IO ()

156

decompressedP :: Producer ByteString IO (Either (Producer ByteString IO ()) ())

next decompressedP :: IO (Either (Either (Producer ByteString IO ()) ()) (ByteString, Producer ByteString IO (Either (Producer ByteString IO ()) ())))

[12,177,366,524,703,855,1017,1197,1378,1542,1730,1896,2060,2285,2484,2601,2633,2828,3014,3196,3362,3531,3694,3876,4090,4274,4443,4624,4695,4868,5044,5207,5376,5597,5756,5941,6122,6319,6493,6651,6840,7012,7201,7382,7454,7475,7654,7726,7799,7977,8144,8323,8488,8650,8828,9001,9183,9365,9559,9741,9902,10069,10241,10399,10580,10651,10846,11032,11219,11397,11562,11744,11935,12011,12083,12281,12356,12429,12499,12734,12827,12907,13080,13258,13328,13550,13628,13801,13887,14047,14239,14450,14528,14749,14826,14899,14927,15008,15223,15300,15382,15594,15672,15744,15926,15958,16141,16335,16406,16477,16656,16728,16916,16987,17010,17096,17255,17415,17587,17773,17997,18075,18147,18337,18541,18619,18696,18775,18869,19036,19197,19368,19548,19732,19803,19826,19850,20074,20152,20235,20260,20441,20621,20839,20916,20986,21205,21283,21354,21573,21651,21725,21820,21892,22070,22142,22308,22492,22670,22847,23008,23183,23354,23524,23696,23917,23995,24110,24141,24228,24448,24526,24611,24822,24899,24982,25194,25272

UnResolved (OfsDelta 424 (PackCompressed {packCLevel = DefaultCompression, packCContent = Delta 338 338 [InsertInstruction "tree e3da1bc442931d8e278c7c1fe384a59ef4ca6d44",CopyInstruction 45 56,InsertInstruction "efb7ac8f19284379adf00e269a28dde6071a0053",CopyInstruction 141 51,InsertInstruction "269645",CopyInstruction 198 60,InsertInstruction "269645",CopyInstruction 264 74]}))

[12,177,366,524,703,855,1017,1197,1378,1542,1730,1896,2060,2285,2484,2601,2633,2828,3014,3196,3362,3531,3694,3876,4090,4274,4443,4624,4695,4868,5044,5207,5376,5597,5756,5941,6122,6319,6493,6651,6840,7012,7201,7382,7454,7475,7654,7726,7799,7977,8144,8323,8488,8650,8828,9001,9183,9365,9559,9741,9902,10069,10241,10399,10580,10651,10846,11032,11219,11397,11562,11744,11935,12011,12083,12281,12356,12429,12499,12734,12827,12907,13080,13258,13328,13550,13628,13801,13887,14047,14239,14450,14528,14749,14826,14899,14927,15008,15223,15300,15382,15594,15672,15744,15926,15958,16141,16335,16406,16477,16656,16728,16916,16987,17010,17096,17255,17415,17587,17773,17997,18075,18147,18337,18541,18619,18696,18775,18869,19036,19197,19368,19548,19732,19803,19826,19850,20074,20152,20235,20260,20441,20621,20839,20916,20986,21205,21283,21354,21573,21651,21725,21820,21892,22070,22142,22308,22492,22670,22847,23008,23183,23354,23524,23696,23917,23995,24110,24141,24228,24448,24526,24611,24822,24899,24982,25194,25272

True

In [4]:
entries <- indexPackfile ".git/objects/pack/pack-23a94077d4edc73dd898189c8fa17a49c5e3ad23.pack"
M.size entries

user error (Pattern match failure in do expression at src/Duffer/Pack/Streaming.hs:57:9-29)